In [1]:
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

# defining the server and the database
server = '(localdb)\mssqllocaldb' 
database = 'TSQL_CLASS'  

# Define the connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE ='+ database +';\
    Trusted_Connection=yes;'
) 

### Question 1
### Select the name and price of the cheapest product.

In [14]:
# Thought process 
# Using Product table 
# Use MIN() function to find the cheapest product
# In WHERE Clause implement the MIN() on Price from Product table
# Then query the Name and Price value for the cheapest product
# Format Price field as currency for appearence

query = pd.read_sql_query("""SELECT Prod.Name AS [Product Name], FORMAT(Prod.Price,'C') AS [Cheapest Price]
                             FROM [TSQL_Class].[dbo].[Products] AS Prod
                             WHERE Prod.Price = (SELECT MIN(Prod.Price) FROM [TSQL_Class].[dbo].[Products] AS Prod)""",conn)
query

,Product Name,Cheapest Price
0,Floppy disk,$5.00


### Question 2
### Select the name of each manufacturer along with the
### name and price of its most expensive product


In [59]:
# Thought Process:
# Using Product table and Manufactures table
# [TSQL_Class].[dbo].[Manufactures] AS Mfg
# [TSQL_CLASS].[dbo].[Products] AS Prod
# Use MAX() to get the most expensive product
# Implement MAX() in the subquery on Price column
# Then in the outer query get the Manufacture's name 
# Order by results by Price in descending order

query = pd.read_sql_query("""SELECT Mfg.Name, Prod.Name, FORMAT(Prod.Price,'C')
                             FROM [TSQL_Class].[dbo].[Manufacturers] AS Mfg,
                             [TSQL_CLASS].[dbo].[Products] AS Prod
                             WHERE Prod.Manufacturer = Mfg.Code
                           
                             AND Prod.Price = (SELECT MAX(Prod.Price)
                                               FROM [TSQL_CLASS].[dbo].[Products] AS Prod
                                               WHERE Prod.Manufacturer = Mfg.Code)
                                        
                           ORDER BY (SELECT MAX(Prod.Price)
                                     FROM [TSQL_CLASS].[dbo].[Products] AS Prod
                                     WHERE Prod.Manufacturer = Mfg.Code) DESC """,conn)
query


,Name,Name,
0,Hewlett-Packard,Printer,$270.00
1,Fujitsu,Hard drive,$240.00
2,Sony,Monitor,$240.00
3,Creative Labs,DVD burner,$180.00
4,Creative Labs,DVD drive,$180.00
5,Iomega,ZIP drive,$150.00
6,Winchester,Memory,$120.00


### Question 3
### Select the name and last name of employees working
### for departments with a budget greater than 60,000 dollars.

In [60]:
# Thought Proces:
# Using tables [TSQL_CLASS].[dbo].[Departments] AS Dept
# AND [TSQL_CLASS].[dbo].[Employees] AS Emp
# ON Emp.Department = Dept.Code
# Dept.Budget > 60,000
# Outer query SELECT employee Name and LastName fields
# Order BY employee name

query = pd.read_sql_query("""SELECT Emp.Name AS [First Name], 
                             Emp.LastName AS [Last Name],
                             Dept.Name
                             FROM [TSQL_CLASS].[dbo].[Departments] AS Dept,
                             [TSQL_CLASS].[dbo].[Employees] AS Emp
                             WHERE Emp.Department = Dept.Code
                             
                             AND Dept.Budget = (SELECT Dept.Budget
                                                FROM [TSQL_CLASS].[dbo].[Departments] AS Dept
                                                WHERE Emp.Department = Dept.Code
                                                AND Dept.Budget > 60000)
                                                
                             ORDER BY Emp.Name ASC""",conn)
query

,First Name,Last Name,Name
0,Elizabeth,Doe,IT
1,Eric,Goldsmith,Human Resources
2,Kate,Doe,Human Resources
3,Kumar,Swamy,IT
4,Mary-Anne,Foster,IT
5,Michael,Rogers,IT
6,Sasi,Scott,IT
7,Zacary,Efron,Human Resources


### Question 4
### Select the departments with a budget larger than the
### average budget of all the departments.


In [82]:
# Thought Process:
# Using tables [TSQL_CLASS].[dbo].[Departments] AS Dept
# Using the outer SELECT statement to get Code, Name and Budget columns
# Using the inner SELECT statement to filter budgets with AVG() function 
# ON Dept.Budget > AVG(Dept.Budget)

query = pd.read_sql_query("""SELECT Dept.Code,
                             Dept.Name,
                             FORMAT(Dept.Budget,'C')
                             FROM [TSQL_CLASS].[dbo].[Departments] AS Dept
                             WHERE Dept.Budget > (SELECT AVG(Dept.Budget)
                                                  FROM [TSQL_CLASS].[dbo].[Departments] AS Dept)
                             ORDER BY Dept.Budget""",conn)
query


,Code,Name,
0,59,Human Resources,"$240,000.00"


### Question 5
### Select the names of departments with more than two employees.

In [13]:
# Thought Process:
# Using SELECT statement to get Emp.Name column
# FROM tables [TSQL_CLASS].[dbo].[Employees] AS Emp AND [TSQL_CLASS].[dbo].[Departments] AS Dept
# WHERE Emp.Department = Dept.Code
# AND  Dept.Code = (SELECT  Dept.Code FROM Dept table WHERE COUNT(Emp.SSN) > 2)


query = pd.read_sql_query("""SELECT Dept.Name AS Department, 
                             COUNT(Emp.SSN) AS [Number of Employees]
                             FROM [TSQL_CLASS].[dbo].[Departments] AS Dept,
                             [TSQL_CLASS].[dbo].[Employees] AS Emp
                             WHERE Emp.Department = Dept.Code
                             
                             AND Dept.Code IN (SELECT Emp.Department
                                              FROM [TSQL_CLASS].[dbo].[Employees] AS Emp
                                              GROUP BY Emp.Department 
                                              HAVING COUNT(Emp.SSN) > 2)
                                              
                            GROUP BY Dept.Name""",conn)
query

,Department,Number of Employees
0,Human Resources,3
1,IT,5


### Question 6
### Select the codes of all warehouses that are saturated (a
### warehouse is saturated if the number of boxes in it is larger than the warehouse's capacity)

In [20]:
# Thought Process: 
# Using table [TSQL_CLASS].[dbo].[Boxes] AS Box AND [TSQL_CLASS].[dbo].[Warehouses] AS Whrs
# ON Box.Wharehouse = Whrs.Code
# Whrs.Code IN HAVING (COUNT(Box.Warehouse) > Whrs.Capacity)

query = pd.read_sql_query("""SELECT Whrs.Code, Whrs.Location
                             FROM [TSQL_CLASS].[dbo].[Boxes] AS Box,
                             [TSQL_CLASS].[dbo].[Warehouses] AS Whrs
                             WHERE Box.Warehouse = Whrs.Code
                             
                             AND Whrs.Code IN (SELECT Box.Warehouse
                                               FROM [TSQL_CLASS].[dbo].[Boxes] AS Box
                                               GROUP BY Box.Warehouse
                                               HAVING COUNT(Box.Warehouse) > Whrs.Capacity)
                                               
                            GROUP BY Whrs.Code,Whrs.Location""",conn)
query

,Code,Location
0,1,Chicago


### Question 7
### Select the codes of all the boxes located in Chicago.


In [24]:
# Thought Process:
# Using SELECT in Outer query to get Boxes table Code field
# FROM [TSQL_CLASS].[dbo].[Boxes] AS Box AND [TSQL_CLASS].[dbo].[Warehouses] AS Whrs
# ON Box.Warehouse = Whrs.Code
# Use Box.Warehouse to get access to the location
# In the subquery USE [TSQL_CLASS].[dbo].[Warehouses] AS Whrs table
# Using WHERE clause to filter location for Chicago

query = pd.read_sql_query("""SELECT Whrs.Location, Box.Code AS [Product Code],
                             FORMAT(Box.Value,'C')
                             FROM [TSQL_CLASS].[dbo].[Boxes] AS Box,
                             [TSQL_CLASS].[dbo].[Warehouses] AS Whrs
                             WHERE Box.Warehouse = Whrs.Code
                             
                             AND Box.Warehouse IN (SELECT Whrs.Code
                             FROM [TSQL_CLASS].[dbo].[Warehouses] AS Whrs
                             WHERE Whrs.Location = 'Chicago')""",conn)
query

,Location,Product Code,
0,Chicago,4H8P,$250.00
1,Chicago,7G3H,$200.00
2,Chicago,8JN6,$75.00
3,Chicago,9J6F,$175.00
4,Chicago,P0H6,$125.00
5,Chicago,P2T6,$150.00


### Question 8
### Select the name of pieces provided by provider with code "HAL".


In [29]:
# Thought Process: 
# JOIN three table [TSQL_CLASS].[dbo].[Pieces] AS Pcs, [TSQL_CLASS].[dbo].[Provides] AS Provides
# AND [TSQL_CLASS].[dbo].[Providers] AS Providers
# ON Pcs.Code = Provides.Piece AND Provides.Provider = Providers.Code
# Subquery SELECT filter provider = 'HAL'
# Outer query SELECT Pcs.Name provided by 'HAL'

query = pd.read_sql_query("""SELECT Provides.Provider AS [Provider Code], 
                             Providers.Name AS [Provider], 
                             Pcs.Name AS [Piece Name]
                             FROM [TSQL_CLASS].[dbo].[Pieces] AS Pcs,
                             [TSQL_CLASS].[dbo].[Provides] AS Provides,
                             [TSQL_CLASS].[dbo].[Providers] AS Providers
                             WHERE Pcs.Code = Provides.Piece 
                             AND Provides.Provider = Providers.Code
                             
                             AND Provides.Provider = (SELECT Providers.Code
                                                      FROM [TSQL_CLASS].[dbo].[Providers] AS Providers
                                                      WHERE Providers.Code = 'HAL')
                                                      
                            ORDER BY Pcs.Name""",conn)
query


,Provider Code,Provider,Piece Name
0,HAL,Clarke Enterprises,Bolt
1,HAL,Clarke Enterprises,Screw
2,HAL,Clarke Enterprises,Sprocket


### Question 9
### For each piece, find the most expensive offering of that
### piece and include the piece name, provider name, and price
### (note that there could be two providers who supply the same piece at the most expensive price).

In [42]:
# Thought Process:
# JOIN three table [TSQL_CLASS].[dbo].[Pieces] AS Pcs, [TSQL_CLASS].[dbo].[Provides] AS Provides
# AND [TSQL_CLASS].[dbo].[Providers] AS Providers
# ON Pcs.Code = Provides.Piece AND Provides.Provider = Providers.Code
# IN the subquery using WHERE clause to compare Provides.Piece to Pcs.Code 
# Keeping in mind that there may be more than one provider per piece
# Using SELECT to get MAX() value for Price
# OUTER query SELECT get Provider code, Provider Name, Piece Name
# AND Piece Price


query = pd.read_sql_query("""SELECT Provides.Provider AS [Provider Code], 
                             Providers.Name AS [Provider], 
                             Pcs.Name AS [Piece Name],
                             FORMAT(Provides.Price,'C')
                             FROM [TSQL_CLASS].[dbo].[Pieces] AS Pcs,
                             [TSQL_CLASS].[dbo].[Provides] AS Provides,
                             [TSQL_CLASS].[dbo].[Providers] AS Providers
                             WHERE Pcs.Code = Provides.Piece 
                             AND Provides.Provider = Providers.Code
                             
                             AND Provides.Price IN (SELECT MAX(Provides.Price)
                                                   FROM [TSQL_CLASS].[dbo].[Provides] AS Provides
                                                   WHERE Provides.Piece = Pcs.Code)
                                                   
                             ORDER BY Provides.Provider,Provides.Price """,conn)
query


,Provider Code,Provider,Piece Name,
0,HAL,Clarke Enterprises,Screw,$20.00
1,RBT,Susan Calvin Corp.,Bolt,$7.00
2,RBT,Susan Calvin Corp.,Sprocket,$15.00
3,RBT,Susan Calvin Corp.,Nut,$50.00
